# Obtencion, limpieza, reetiquetado

In [1]:
from trackml.dataset import load_event
hits, cells, particles, truth = load_event('train_100_events/event000001000')

In [2]:
pt=(particles['px']**2 + particles['py']**2)**0.5
particles['pt'] = pt

In [3]:
data=hits.merge(truth, on='hit_id', how='left')
data=data[data['particle_id']!=0]
data=data[(data['volume_id']==8) | (data['volume_id']==13) | (data['volume_id']==17)]
data['pt']=(data['tpx']**2 + data['tpy']**2)**0.5

data_0=data.copy()

In [4]:
i=0
for vol in [8, 13, 17]:
    for lay in hits[hits['volume_id']==vol]['layer_id'].unique():
        i=i+1
        data.loc[(data['volume_id'] == vol) & (data['layer_id'] == lay), 'layer_id'] = i

In [5]:
pid=data['particle_id'].unique()

In [6]:
merged_data = hits.merge(truth, on='hit_id', how='inner')
merged_data_0=merged_data[merged_data['particle_id'] != 0].copy()
merged_data_0['pt']=(merged_data_0['tpx']**2 + merged_data_0['tpy']**2)**0.5

In [7]:
pid_075=particles[particles['pt']>0.75]['particle_id'].unique()

In [8]:
data_075=data[data['particle_id'].isin(pid_075)].copy()

In [49]:
data_075H=data_075[data_075['pt']>0.75].copy()
#data_075H.to_csv('data_075H.csv', index=False)

In [38]:
import plotly.express as px

fig = px.scatter_3d(
    data_075,
    x='x',
    y='y',
    z='z',
    color='layer_id',
    size_max=2
)

# Mostrar la figura
fig.show()

# Formacion de tripletes

En el siguiente script de python que realicé, implemento la formación y el filtrado de tripletes entre hits de capas adyacentes (estos datos estan en el pandas dataframe "data", el cual  tiene las columnas: 'hit_id', 'x', 'y', 'z', 'volume_id', 'layer_id', 'module_id', 'particle_id'). El filtrado de tripletes sigue varios criterios como puedes ver en el código. Quiero que añadas el f

In [ ]:
import math 
import numpy as np

def cilindricas(x, y, z, grados=False):
    # Calcular ρ (distancia radial)
    rho = math.sqrt(x**2 + y**2)
    
    # Calcular φ (ángulo azimutal)
    phi = math.atan2(y, x)
    
    # Asegurar que φ esté en el rango [0, 2π)
    if phi < 0:
        phi += 2 * math.pi
    
    # Convertir a grados si se solicita
    if grados:
        phi = math.degrees(phi)
    
    return (rho, phi, z)

tripletes = []
i=0

for l in range(1,9):
    for hit1 in data[data['layer_id']==l].itertuples():
        for hit2 in data[data['layer_id']==l+1].itertuples():
        
            r1,phi1,z1=cilindricas(hit1.x,hit1.y,hit1.z)
            r2,phi2,z2=cilindricas(hit2.x,hit2.y,hit2.z)
                
            #filtro del primer duplete
            if hit2.x-hit1.x !=0:
                z0=z1-hit1.x*(z2-z1)/(hit2.x-hit1.x)
            else:
                z0=1000  #asigno un valor grande para que no pase el filtro
            if abs((phi2-phi1)/(r2-r1))<=6*pow(10,4) and abs(z0)<=100:
                for hit3 in data[data['layer_id']==l+2].itertuples():
                    r3,phi3,z3=cilindricas(hit3.x,hit3.y,hit3.z)

                    #filtro del segundo duplete
                    if hit3.x-hit2.x !=0:
                        z00=z2-hit2.x*(z3-z2)/(hit3.x-hit2.x)
                    else:
                        z00=1000  #asigno un valor grande para que no pase el filtro                    
                    
                    if abs((phi3-phi2)/(r3-r2))<=6*pow(10,4) and abs(z00)<=100:
                        #parametros de tripletes

                        #theta break
                        v_12=np.array([hit2.x-hit1.x,hit2.y-hit1.y,hit2.z-hit1.z])
                        v_23=np.array([hit3.x-hit2.x,hit3.y-hit2.y,hit3.z-hit2.z])
                        cos_theta=np.dot(v_12,v_23)/(np.linalg.norm(v_12)*np.linalg.norm(v_23))
                        theta_break=math.acos(cos_theta)

                        #phi break
                        f_12=np.array([z2-z1,r2-r1])
                        f_23=np.array([z3-z2,r3-r2])
                        cos_phi=np.dot(f_12,f_23)/(np.linalg.norm(f_12)*np.linalg.norm(f_23))
                        phi_break=math.acos(cos_phi)

                        if (theta_break<=0.05) and (phi_break<=0.06):

                            
                            label=0
                            if (hit1.particle_id==hit2.particle_id) and (hit2.particle_id==hit3.particle_id):
                                label=1
                            tripletes.append({
                                'r1': r1, 'phi1': phi1, 'z1': z1,
                                'r2': r2, 'phi2': phi2, 'z2': z2,
                                'r3': r3, 'phi3': phi3, 'z3': z3,
                                'theta_break': theta_break,
                                'phi_break': phi_break,
                                'label': label
                            })
                            i=i+1
                            if i%5000==0: print(f'Capa: {l} | Tripletes totales: {i}')
                            

                        

   

In [ ]:
import math

def circle_from_3points_xy(p1, p2, p3, eps=1e-12):
    """
    Calcula el círculo que pasa por tres puntos en el plano (x, y).

    p1, p2, p3 : (x, y) en las MISMAS unidades (aquí: mm).
    Devuelve centro y radio en esas mismas unidades.
    """
    (x1, y1) = p1
    (x2, y2) = p2
    (x3, y3) = p3

    D = 2.0 * (
        x1 * (y2 - y3) +
        x2 * (y3 - y1) +
        x3 * (y1 - y2)
    )

    if abs(D) < eps:
        raise ValueError("Los tres puntos son colineales o casi colineales; no definen un círculo único.")

    x1_sq_y1_sq = x1**2 + y1**2
    x2_sq_y2_sq = x2**2 + y2**2
    x3_sq_y3_sq = x3**2 + y3**2

    xc = (
        x1_sq_y1_sq * (y2 - y3) +
        x2_sq_y2_sq * (y3 - y1) +
        x3_sq_y3_sq * (y1 - y2)
    ) / D

    yc = (
        x1_sq_y1_sq * (x3 - x2) +
        x2_sq_y2_sq * (x1 - x3) +
        x3_sq_y3_sq * (x2 - x1)
    ) / D

    R = math.hypot(x1 - xc, y1 - yc)  # mismo unit que x,y (mm)

    return xc, yc, R


def pt_from_3hits_mm(hit1, hit2, hit3, B_T, q_sign=1.0):
    """
    Calcula pT (GeV/c) a partir de 3 hits en mm y B en tesla.

    hit1, hit2, hit3 : (x_mm, y_mm, z_mm)
        Posiciones de los hits en milímetros.
        Solo se usan x,y para pT.
    B_T : float
        Campo magnético en tesla.
    q_sign : float
        Signo de la carga en unidades de e (+1 o -1 típicamente).

    Devuelve
    --------
    pt_GeV : float
        Momento transverso en GeV/c (módulo).
    R_m : float
        Radio de curvatura en metros.
    """
    x1, y1, _ = hit1
    x2, y2, _ = hit2
    x3, y3, _ = hit3

    # Círculo en mm
    _, _, R_mm = circle_from_3points_xy((x1, y1), (x2, y2), (x3, y3))

    # Convertir a metros
    R_m = R_mm / 1000.0

    # Fórmula pT[GeV/c] ≈ 0.3 * |q/e| * B[T] * R[m]
    pt_GeV = 0.3 * abs(q_sign) * B_T * R_m

    return pt_GeV, R_m


# Ejemplo de uso
if __name__ == "__main__":
    # Tres hits en mm
    h1 = (0.0, 100.0, 0.0)
    h2 = (50.0, 110.0, 100.0)
    h3 = (100.0, 150.0, 200.0)
    B = 1.002  # tesla
    pt, R = pt_from_3hits_mm(h1, h2, h3, B, q_sign=+1)
    print(f"pT = {pt:.3f} GeV/c, R = {R:.4f} m")


pT = 0.139 GeV/c, R = 0.1217 m


In [21]:
h1 = (-46.529701, 106.167999, -380.192993)
h2 = (-58.022099, 161.100006, -354.661011)
h3 = (-50.431900, 252.209000, -312.000000)
B = 1.003  # tesla
pt, R = pt_from_3hits_mm(h1, h2, h3, B, q_sign=+1)
print(f"pT = {pt:.3f} GeV/c, R = {R:.4f} m")

pT = 0.077 GeV/c, R = 0.2560 m


In [13]:
particles[particles['particle_id']==pid[10]]

,particle_id,vx,vy,vz,px,py,pz,q,nhits
4323,301761790893625345,18.5753,-3.99472,-441.815002,-0.11529,0.121036,0.081687,1,10


Lo mismo que lo anterior pero con try

In [ ]:
import math 
import numpy as np
import pandas as pd

data = pd.read_csv('entrada.csv')
def cilindricas(x, y, z, grados=False):
    # Calcular ρ (distancia radial)
    rho = math.sqrt(x**2 + y**2)
    
    # Calcular φ (ángulo azimutal)
    phi = math.atan2(y, x)
    
    # Asegurar que φ esté en el rango [0, 2π)
    if phi < 0:
        phi += 2 * math.pi
    
    # Convertir a grados si se solicita
    if grados:
        phi = math.degrees(phi)
    
    return (rho, phi, z)

df = pd.DataFrame(columns=['r1', 'phi1', 'z1', 'r2', 'phi2', 'z2','r3', 'phi3', 'z3', 'theta_break', 'phi_break', 'label'])
i=0
label=0

for l in range(1,9):
    for hit1 in data[data['layer_id']==l].itertuples():
        for hit2 in data[data['layer_id']==l+1].itertuples():
        
            r1,phi1,z1=cilindricas(hit1.x,hit1.y,hit1.z)
            r2,phi2,z2=cilindricas(hit2.x,hit2.y,hit2.z)
                
            #filtro del primer duplete
            if hit2.x-hit1.x !=0:
                z0=z1-hit1.x*(z2-z1)/(hit2.x-hit1.x)
            else:
                z0=1000  #asigno un valor grande para que no pase el filtro
            if abs((phi2-phi1)/(r2-r1))<=6*pow(10,4) and abs(z0)<=100:
                for hit3 in data[data['layer_id']==l+2].itertuples():
                    r3,phi3,z3=cilindricas(hit3.x,hit3.y,hit3.z)

                    #filtro del segundo duplete
                    if hit3.x-hit2.x !=0:
                        z00=z2-hit2.x*(z3-z2)/(hit3.x-hit2.x)
                    else:
                        z00=1000  #asigno un valor grande para que no pase el filtro                    
                    
                    if abs((phi3-phi2)/(r3-r2))<=6*pow(10,4) and abs(z00)<=100:
                        #parametros de tripletes

                        #theta break
                        v_12=np.array([hit2.x-hit1.x,hit2.y-hit1.y,hit2.z-hit1.z])
                        v_23=np.array([hit3.x-hit2.x,hit3.y-hit2.y,hit3.z-hit2.z])
                        cos_theta=np.dot(v_12,v_23)/(np.linalg.norm(v_12)*np.linalg.norm(v_23))
                        theta_break=math.acos(cos_theta)

                        #phi break
                        f_12=np.array([z2-z1,r2-r1])
                        f_23=np.array([z3-z2,r3-r2])
                        cos_phi=np.dot(f_12,f_23)/(np.linalg.norm(f_12)*np.linalg.norm(f_23))
                        phi_break=math.acos(cos_phi)

                        if (theta_break<=0.05) and (phi_break<=0.05):
                                                      
                            if (hit1.particle_id==hit2.particle_id) and (hit2.particle_id==hit3.particle_id):
                                label=1
                            else:
                                label=0

                            df.loc[i] = [r1, phi1, z1, r2, phi2, z2, r3, phi3, z3, theta_break, phi_break, label]   
                                
                            i=i+1
                            if i%20000==0: print(f'Capa: {l} | Tripletes totales: {i}')
                            
                                
df.to_csv('resultado.csv')
   

In [7]:
import plotly.express as px

fig = px.scatter_3d(
    data,
    x='x',
    y='y',
    z='z',
    color='layer_id',
    size_max=2
)

# Mostrar la figura
fig.show()

## Tripletes aleatorios

Tripletes correctos:

In [10]:
import pandas as pd
import math 
import numpy as np

Modificado con filtro > 0.75

In [11]:
#tripletes_correctos = pd.DataFrame(columns=['r1', 'phi1', 'z1', 'r2', 'phi2', 'z2','r3', 'phi3', 'z3', 'theta_break', 'phi_break', 'label','pid_1','pid_2','pid_3'])
data=data_075.copy()

tripletes_correctos = pd.DataFrame(columns=['r1', 'phi1', 'z1', 'r2', 'phi2', 'z2','r3', 'phi3', 'z3', 'theta_break', 'phi_break', 'label'])
i=0

def cilindricas(x, y, z, grados=False):
    # Calcular ρ (distancia radial)
    rho = math.sqrt(x**2 + y**2)
    
    # Calcular φ (ángulo azimutal)
    phi = math.atan2(y, x)
    
    # Asegurar que φ esté en el rango [0, 2π)
    if phi < 0:
        phi += 2 * math.pi
    
    # Convertir a grados si se solicita
    if grados:
        phi = math.degrees(phi)
    
    return (rho, phi, z)

for pid in data['particle_id'].unique():
    particulas=data[data['particle_id']==pid]
    n_hits=particulas.shape[0]
    if n_hits>=3 and n_hits==merged_data_0[merged_data_0['particle_id']==pid].shape[0]:
        for h in range(n_hits-2):
            r1,phi1,z1=cilindricas(particulas.iloc[h].x,particulas.iloc[h].y,particulas.iloc[h].z)
            r2,phi2,z2=cilindricas(particulas.iloc[h+1].x,particulas.iloc[h+1].y,particulas.iloc[h+1].z)
            r3,phi3,z3=cilindricas(particulas.iloc[h+2].x,particulas.iloc[h+2].y,particulas.iloc[h+2].z)
            
            #primer duplete
            if particulas.iloc[h+1].x-particulas.iloc[h].x !=0:
                z0=z1-particulas.iloc[h].x*(z2-z1)/(particulas.iloc[h+1].x-particulas.iloc[h].x)
            else:
                z0=1000

            delta1=abs((phi2-phi1)/(r2-r1))

            #segundo duplete
        
            if particulas.iloc[h+2].x-particulas.iloc[h+1].x !=0:
                z00=z2-particulas.iloc[h+1].x*(z3-z2)/(particulas.iloc[h+2].x-particulas.iloc[h+1].x)
            else:
                z00=1000

            delta2=abs((phi3-phi2)/(r3-r2))

            #parametros de tripletes
            #theta break
            v_12=np.array([particulas.iloc[h+1].x-particulas.iloc[h].x,particulas.iloc[h+1].y-particulas.iloc[h].y,particulas.iloc[h+1].z-particulas.iloc[h].z])
            v_23=np.array([particulas.iloc[h+2].x-particulas.iloc[h+1].x,particulas.iloc[h+2].y-particulas.iloc[h+1].y,particulas.iloc[h+2].z-particulas.iloc[h+1].z])
            cos_theta=np.dot(v_12,v_23)/(np.linalg.norm(v_12)*np.linalg.norm(v_23))
            theta_break=math.acos(cos_theta)    
            #phi break
            f_12=np.array([z2-z1,r2-r1])
            f_23=np.array([z3-z2,r3-r2])
            cos_phi=np.dot(f_12,f_23)/(np.linalg.norm(f_12)*np.linalg.norm(f_23))
            phi_break=math.acos(cos_phi)

            if delta1<=6*pow(10,4) and delta2<=6*pow(10,4) and z0<=100 and z00<=100 and theta_break<=0.05 and phi_break<=0.05:
                #tripletes_correctos.loc[i] = [r1, phi1, z1, r2, phi2, z2, r3, phi3, z3, theta_break,phi_break,1,particulas.iloc[h].particle_id,particulas.iloc[h+1].particle_id,particulas.iloc[h+2].particle_id]
                tripletes_correctos.loc[i] = [r1, phi1, z1, r2, phi2, z2, r3, phi3, z3, theta_break,phi_break,1]
                i=i+1
                print(f'N: {i} |Particle ID: {pid} | Numero de hits: {n_hits}')


N: 1 |Particle ID: 968283952928260096 | Numero de hits: 3
N: 2 |Particle ID: 693557503710986240 | Numero de hits: 7
N: 3 |Particle ID: 693557503710986240 | Numero de hits: 7
N: 4 |Particle ID: 4517687120101376 | Numero de hits: 3
N: 5 |Particle ID: 671037718867738624 | Numero de hits: 6
N: 6 |Particle ID: 671037718867738624 | Numero de hits: 6
N: 7 |Particle ID: 671037718867738624 | Numero de hits: 6
N: 8 |Particle ID: 711571558623084544 | Numero de hits: 3
N: 9 |Particle ID: 364798922801020928 | Numero de hits: 5
N: 10 |Particle ID: 364798922801020928 | Numero de hits: 5
N: 11 |Particle ID: 364798922801020928 | Numero de hits: 5
N: 12 |Particle ID: 337773339307147264 | Numero de hits: 8
N: 13 |Particle ID: 337773339307147264 | Numero de hits: 8
N: 14 |Particle ID: 337773339307147264 | Numero de hits: 8
N: 15 |Particle ID: 337773339307147264 | Numero de hits: 8
N: 16 |Particle ID: 337773339307147264 | Numero de hits: 8
N: 17 |Particle ID: 364803045969625088 | Numero de hits: 5
N: 18 |P

In [12]:
tripletes_correctos.to_csv('TC_1.csv', index=False)

In [ ]:
data_075.to_csv('Preprocesamiento\data_075.csv', index=False)